In [1]:
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain.schema.document import Document
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores.chroma import Chroma

In [2]:
chroma_path = "../chroma"

In [3]:
vectordb = Chroma(persist_directory=chroma_path, embedding_function=SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2"))

/opt/homebrew/Caskroom/miniforge/base/envs/pmra/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/Caskroom/miniforge/base/envs/pmra/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
retriever = vectordb.as_retriever()

In [5]:
URI = "https://repro-toxicity-mvdiw.eastus2.inference.ml.azure.com/score"
KEY = "jAYo2rLdFte9ZF35LanyBXHE9YUW072P"

In [6]:
from langchain_community.chat_models.azureml_endpoint import AzureMLChatOnlineEndpoint, CustomOpenAIChatContentFormatter
from langchain_community.llms.azureml_endpoint import AzureMLEndpointApiType

llm = AzureMLChatOnlineEndpoint(
    endpoint_url=URI,
    endpoint_api_type=AzureMLEndpointApiType.dedicated,
    endpoint_api_key=KEY,
    content_formatter=CustomOpenAIChatContentFormatter(),
)

response = llm.invoke("Will humans ever solve the Collatz conjecture?")
print(response)

content='The eternal question!\n\nThe Collatz Conjecture, also known as the' type='assistant' id='run-47b4ab0c-d1a7-4461-a5bb-d3f5c85759e7-0'


In [7]:
question = "What happened with  OECD  Working  Group in  Copenhagen  in  June  1995?"

In [8]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

SYSTEM_TEMPLATE = """
Answer the user's questions based on the below context. 
If the context doesn't contain any relevant information to the question, don't make something up and just say "I don't know":

<context>
{context}
</context>
"""

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SYSTEM_TEMPLATE,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(llm, question_answering_prompt)

In [9]:
from typing import Dict

from langchain_core.runnables import RunnablePassthrough

def parse_retriever_input(params: Dict):
    return params["messages"][-1].content

retrieval_chain = RunnablePassthrough.assign(
    context=parse_retriever_input | retriever,
).assign(
    answer=document_chain,
)

In [10]:
from langchain_core.messages import AIMessage, HumanMessage

query_transform_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="messages"),
        (
            "user",
            "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation. Only respond with the query, nothing else.",
        ),
    ]
)

query_transformation_chain = query_transform_prompt | llm

query_transformation_chain.invoke(
    {
        "messages": [
            HumanMessage(content="Can LangSmith help test my LLM applications?"),
            AIMessage(
                content="Yes, LangSmith can help test and evaluate your LLM applications. It allows you to quickly edit examples and add them to datasets to expand the surface area of your evaluation sets or to fine-tune a model for improved quality or reduced costs. Additionally, LangSmith can be used to monitor your application, log all traces, visualize latency and token usage statistics, and troubleshoot specific issues as they arise."
            ),
            HumanMessage(content="Tell me more!"),
        ],
    }
)

BaseMessage(content='"LLM LangSmith testing and evaluation"', type='assistant', id='run-b60ebabf-6c74-43f7-b23b-2f1a5da7629a-0')

In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch

query_transforming_retriever_chain = RunnableBranch(
    (
        lambda x: len(x.get("messages", [])) == 1,
        # If only one message, then we just pass that message's content to retriever
        (lambda x: x["messages"][-1].content) | retriever,
    ),
    # If messages, then we pass inputs to LLM chain to transform the query, then pass to retriever
    query_transform_prompt | llm | StrOutputParser() | retriever,
).with_config(run_name="chat_retriever_chain")

In [12]:
SYSTEM_TEMPLATE = """
Answer the user's questions based on the below context. 
If the context doesn't contain any relevant information to the question, don't make something up and just say "I don't know":

<context>
{context}
</context>
"""

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SYSTEM_TEMPLATE,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(llm, question_answering_prompt)

conversational_retrieval_chain = RunnablePassthrough.assign(
    context=query_transforming_retriever_chain,
).assign(
    answer=document_chain,
)

In [13]:
response = conversational_retrieval_chain.invoke(
    {
        "messages": [
            HumanMessage(content=question),
        ]
    }
)

In [15]:
response

{'messages': [HumanMessage(content='What happened with  OECD  Working  Group in  Copenhagen  in  June  1995?')],
 'context': [Document(page_content='The OECD initiated a high -priority activity in 1998 to revise e xisting guidelines and to develop', metadata={'id': '../documents/OECD TG 441 (2009).pdf:0:2', 'page': 0, 'source': '../documents/OECD TG 441 (2009).pdf'}),
  Document(page_content='A final report sh ould include:', metadata={'id': '../documents/OECD TG 441 (2009).pdf:11:5', 'page': 11, 'source': '../documents/OECD TG 441 (2009).pdf'}),
  Document(page_content='416 OECD/OCDE\n4/13Limit test', metadata={'id': '../documents/OECD TG 416 (2001).pdf:3:0', 'page': 3, 'source': '../documents/OECD TG 416 (2001).pdf'}),
  Document(page_content='This Test Guideline is based on those protocols employed in the OECD validation study which', metadata={'id': '../documents/OECD TG 441 (2009).pdf:1:11', 'page': 1, 'source': '../documents/OECD TG 441 (2009).pdf'})],
 'answer': "I don't know. T